In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🧠 Function Approximation and Time Series Modeling\n",
    "\n",
    "**Author:** Your Name  \n",
    "**Date:** 2025-08-07\n",
    "\n",
    "This notebook demonstrates:\n",
    "- 1D function approximation using synthetic data\n",
    "- Time series modeling of a biological dynamic system (Lotka–Volterra equations)\n",
    "- Neural network regression using `scikit-learn`"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 📦 Imports\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from scipy.integrate import solve_ivp\n",
    "from sklearn.neural_network import MLPRegressor\n",
    "from sklearn.preprocessing import StandardScaler\n",
    "from sklearn.metrics import mean_squared_error"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔹 1D Synthetic Data\n",
    "\n",
    "We’ll create a 1D training dataset to use for function approximation."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "x1_data = [-12, -10, -5, -2.5, 2, 4, 6, 7.5]\n",
    "Xtrain_1D = np.array(x1_data).reshape(-1, 1)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🎯 Target Functions\n",
    "\n",
    "We define two functions:\n",
    "- A sinusoidal + quadratic function\n",
    "- A sum of three Gaussians"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Sinusoidal + Quadratic function\n",
    "def Sin_Square_1D(X, noise=False):\n",
    "    y = np.sin(X) + (X/5)**2\n",
    "    if noise:\n",
    "        n = np.random.normal(0, 0.03, X.shape[0])\n",
    "        return (y[:, 0] * (1 + n))\n",
    "    return y[:, 0]\n",
    "\n",
    "# Sum of 3 Gaussians\n",
    "def gaus3_func(X, noise=False):\n",
    "    gaus_params = [[1.0, 2.0, -8.0], [1.5, 1.5, 0.0], [1.4, 1.0, 5.0]]\n",
    "    result = np.zeros(X.shape[0])\n",
    "    for a, std, mean in gaus_params:\n",
    "        result += a / (std * 2.5) * np.exp(-0.5 * ((X[:, 0] - mean)/std)**2)\n",
    "    if noise:\n",
    "        result += np.random.normal(0, 0.01, X.shape[0])\n",
    "    return -result"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📊 Visualize the Functions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "Ytrain_SS1D = Sin_Square_1D(Xtrain_1D)\n",
    "Ytrain_3G = gaus3_func(Xtrain_1D)\n",
    "\n",
    "# Plot\n",
    "plt.figure(figsize=(10, 4))\n",
    "plt.subplot(1, 2, 1)\n",
    "plt.title(\"Sin + Square Function\")\n",
    "plt.plot(Xtrain_1D, Ytrain_SS1D, 'bo-')\n",
    "plt.grid(True)\n",
    "\n",
    "plt.subplot(1, 2, 2)\n",
    "plt.title(\"Sum of 3 Gaussians\")\n",
    "plt.plot(Xtrain_1D, Ytrain_3G, 'ro-')\n",
    "plt.grid(True)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🤖 MLPRegressor: Function Approximation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Scale inputs\n",
    "scaler = StandardScaler()\n",
    "X_scaled = scaler.fit_transform(Xtrain_1D)\n",
    "\n",
    "# Train MLP on Sin_Square_1D\n",
    "mlp = MLPRegressor(hidden_layer_sizes=(20, 20), max_iter=1000, random_state=1)\n",
    "mlp.fit(X_scaled, Ytrain_SS1D)\n",
    "\n",
    "# Predict on a dense grid\n",
    "X_dense = np.linspace(-15, 10, 200).reshape(-1, 1)\n",
    "X_dense_scaled = scaler.transform(X_dense)\n",
    "Y_pred = mlp.predict(X_dense_scaled)\n",
    "Y_true = Sin_Square_1D(X_dense)\n",
    "\n",
    "# Plot\n",
    "plt.figure(figsize=(8, 5))\n",
    "plt.plot(X_dense, Y_true, label=\"True Function\", color='black')\n",
    "plt.plot(X_dense, Y_pred, label=\"MLP Prediction\", linestyle='--')\n",
    "plt.scatter(Xtrain_1D, Ytrain_SS1D, color='red', label=\"Training Points\")\n",
    "plt.title(\"MLP Approximation of Sin + Square\")\n",
    "plt.xlabel(\"x\")\n",
    "plt.ylabel(\"y\")\n",
    "plt.grid(True)\n",
    "plt.legend()\n",
    "plt.show()\n",
    "\n",
    "# Evaluate\n",
    "print(\"MSE:\", mean_squared_error(Y_true, Y_pred))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔄 Lotka–Volterra Predator-Prey Model\n",
    "This models the population dynamics of a prey and predator species over time."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define the equations\n",
    "def lotka_volterra(t, z, alpha=1.5, beta=1.0, delta=0.75, gamma=1.0):\n",
    "    x, y = z\n",
    "    dxdt = alpha * x - beta * x * y\n",
    "    dydt = delta * x * y - gamma * y\n",
    "    return [dxdt, dydt]\n",
    "\n",
    "# Initial conditions and time span\n",
    "z0 = [10, 5]  # initial prey and predator\n",
    "t_span = (0, 15)\n",
    "t_eval = np.linspace(*t_span, 300)\n",
    "sol = solve_ivp(lotka_volterra, t_span, z0, t_eval=t_eval)\n",
    "t = sol.t\n",
    "x, y = sol.y"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot the time series\n",
    "plt.figure(figsize=(8, 5))\n",
    "plt.plot(t, x, label='Prey')\n",
    "plt.plot(t, y, label='Predator')\n",
    "plt.title(\"Predator-Prey Dynamics\")\n",
    "plt.xlabel(\"Time\")\n",
    "plt.ylabel(\"Population\")\n",
    "plt.legend()\n",
    "plt.grid(True)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🧩 Extensions and Ideas\n",
    "\n",
    "- Train a neural network to predict future population given history\n",
    "- Add noise to the data and test robustness\n",
    "- Try Gaussian Process Regression for function learning\n",
    "- Use real ecological data for comparison"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": ""
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}


NameError: name 'null' is not defined

In [ ]:
# 📦 Imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

## 🔹 1D Synthetic Data

We'll create a 1D training dataset to use for function approximation.

In [ ]:
x1_data = [-12, -10, -5, -2.5, 2, 4, 6, 7.5]
Xtrain_1D = np.array(x1_data).reshape(-1, 1)

## 🎯 Target Functions

We define two functions:
- A sinusoidal + quadratic function
- A sum of three Gaussians

In [ ]:
# Sinusoidal + Quadratic function
def Sin_Square_1D(X, noise=False):
    y = np.sin(X) + (X/5)**2
    if noise:
        n = np.random.normal(0, 0.03, X.shape[0])
        return (y[:, 0] * (1 + n))
    return y[:, 0]

# Sum of 3 Gaussians
def gaus3_func(X, noise=False):
    gaus_params = [[1.0, 2.0, -8.0], [1.5, 1.5, 0.0], [1.4, 1.0, 5.0]]
    result = np.zeros(X.shape[0])
    for a, std, mean in gaus_params:
        result += a / (std * 2.5) * np.exp(-0.5 * ((X[:, 0] - mean)/std)**2)
    if noise:
        result += np.random.normal(0, 0.01, X.shape[0])
    return -result

## 📊 Visualize the Functions

In [ ]:
Ytrain_SS1D = Sin_Square_1D(Xtrain_1D)
Ytrain_3G = gaus3_func(Xtrain_1D)

# Plot
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.title("Sin + Square Function")
plt.plot(Xtrain_1D, Ytrain_SS1D, 'bo-')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.title("Sum of 3 Gaussians")
plt.plot(Xtrain_1D, Ytrain_3G, 'ro-')
plt.grid(True)
plt.tight_layout()
plt.show()

## 🤖 MLPRegressor: Function Approximation

In [ ]:
# Scale inputs
scaler = StandardScaler()
X_scaled = scaler.fit_transform(Xtrain_1D)

# Train MLP on Sin_Square_1D
mlp = MLPRegressor(hidden_layer_sizes=(20, 20), max_iter=1000, random_state=1)
mlp.fit(X_scaled, Ytrain_SS1D)

# Predict on a dense grid
X_dense = np.linspace(-15, 10, 200).reshape(-1, 1)
X_dense_scaled = scaler.transform(X_dense)
Y_pred = mlp.predict(X_dense_scaled)
Y_true = Sin_Square_1D(X_dense)

# Plot
plt.figure(figsize=(8, 5))
plt.plot(X_dense, Y_true, label="True Function", color='black')
plt.plot(X_dense, Y_pred, label="MLP Prediction", linestyle='--')
plt.scatter(Xtrain_1D, Ytrain_SS1D, color='red', label="Training Points")
plt.title("MLP Approximation of Sin + Square")
plt.xlabel("x")
plt.ylabel("y")
plt.grid(True)
plt.legend()
plt.show()

# Evaluate
print("MSE:", mean_squared_error(Y_true, Y_pred))

## 🔄 Lotka–Volterra Predator-Prey Model
This models the population dynamics of a prey and predator species over time.

In [ ]:
# Define the equations
def lotka_volterra(t, z, alpha=1.5, beta=1.0, delta=0.75, gamma=1.0):
    x, y = z
    dxdt = alpha * x - beta * x * y
    dydt = delta * x * y - gamma * y
    return [dxdt, dydt]

# Initial conditions and time span
z0 = [10, 5]  # initial prey and predator
t_span = (0, 15)
t_eval = np.linspace(*t_span, 300)
sol = solve_ivp(lotka_volterra, t_span, z0, t_eval=t_eval)
t = sol.t
x, y = sol.y

In [ ]:
# Plot the time series
plt.figure(figsize=(8, 5))
plt.plot(t, x, label='Prey')
plt.plot(t, y, label='Predator')
plt.title("Predator-Prey Dynamics")
plt.xlabel("Time")
plt.ylabel("Population")
plt.legend()
plt.grid(True)
plt.show()

## 🧩 Extensions and Ideas

- Train a neural network to predict future population given history
- Add noise to the data and test robustness
- Try Gaussian Process Regression for function learning
- Use real ecological data for comparison

## 🔮 Time Series Prediction with Neural Networks

Let's implement a neural network to predict future population dynamics based on historical data.

In [ ]:
def create_sequences(data, seq_length):
    """Create sliding window sequences for time series prediction"""
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:(i + seq_length)])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

# Prepare time series data for neural network prediction
seq_length = 20  # Use 20 time steps to predict the next step

# Create sequences for both prey and predator populations
X_prey, y_prey = create_sequences(x, seq_length)
X_pred, y_pred = create_sequences(y, seq_length)

print(f"Prey sequences shape: {X_prey.shape}, targets shape: {y_prey.shape}")
print(f"Predator sequences shape: {X_pred.shape}, targets shape: {y_pred.shape}")

# Split data for training and testing
split_idx = int(0.8 * len(X_prey))
X_prey_train, X_prey_test = X_prey[:split_idx], X_prey[split_idx:]
y_prey_train, y_prey_test = y_prey[:split_idx], y_prey[split_idx:]

In [ ]:
# Train MLPRegressor for time series prediction
scaler_ts = StandardScaler()
X_prey_train_scaled = scaler_ts.fit_transform(X_prey_train)
X_prey_test_scaled = scaler_ts.transform(X_prey_test)

# Create and train the neural network
mlp_ts = MLPRegressor(
    hidden_layer_sizes=(50, 30, 10),
    max_iter=2000,
    random_state=42,
    learning_rate_init=0.001,
    early_stopping=True,
    validation_fraction=0.1
)

mlp_ts.fit(X_prey_train_scaled, y_prey_train)

# Make predictions
y_pred_train = mlp_ts.predict(X_prey_train_scaled)
y_pred_test = mlp_ts.predict(X_prey_test_scaled)

# Calculate metrics
train_mse = mean_squared_error(y_prey_train, y_pred_train)
test_mse = mean_squared_error(y_prey_test, y_pred_test)

print(f"Training MSE: {train_mse:.4f}")
print(f"Testing MSE: {test_mse:.4f}")
print(f"Model R² Score: {mlp_ts.score(X_prey_test_scaled, y_prey_test):.4f}")

In [ ]:
# Visualize the predictions
plt.figure(figsize=(12, 8))

# Plot 1: Training predictions
plt.subplot(2, 2, 1)
plt.plot(y_prey_train[:50], label='True Values', alpha=0.7)
plt.plot(y_pred_train[:50], label='Predictions', alpha=0.7)
plt.title('Training Predictions (First 50 points)')
plt.legend()
plt.grid(True)

# Plot 2: Test predictions
plt.subplot(2, 2, 2)
plt.plot(y_prey_test, label='True Values', alpha=0.7)
plt.plot(y_pred_test, label='Predictions', alpha=0.7)
plt.title('Test Predictions')
plt.legend()
plt.grid(True)

# Plot 3: Prediction vs True scatter plot
plt.subplot(2, 2, 3)
plt.scatter(y_prey_test, y_pred_test, alpha=0.6)
plt.plot([y_prey_test.min(), y_prey_test.max()], 
         [y_prey_test.min(), y_prey_test.max()], 'r--', lw=2)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.title('Prediction vs True Values')
plt.grid(True)

# Plot 4: Residuals
plt.subplot(2, 2, 4)
residuals = y_prey_test - y_pred_test
plt.scatter(y_pred_test, residuals, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predictions')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.grid(True)

plt.tight_layout()
plt.show()

## 🔊 Noise Robustness Testing

Let's test how robust our models are to different levels of noise in the data.

In [ ]:
# Test robustness with different noise levels
noise_levels = [0.0, 0.05, 0.1, 0.15, 0.2]
mse_scores = []

plt.figure(figsize=(15, 10))

for i, noise_level in enumerate(noise_levels):
    # Add noise to the original functions
    Ytrain_SS1D_noisy = Sin_Square_1D(Xtrain_1D, noise=False)
    if noise_level > 0:
        noise = np.random.normal(0, noise_level, Ytrain_SS1D_noisy.shape)
        Ytrain_SS1D_noisy += noise
    
    # Train model with noisy data
    scaler_noise = StandardScaler()
    X_scaled_noise = scaler_noise.fit_transform(Xtrain_1D)
    
    mlp_noise = MLPRegressor(hidden_layer_sizes=(20, 20), max_iter=1000, random_state=1)
    mlp_noise.fit(X_scaled_noise, Ytrain_SS1D_noisy)
    
    # Test on clean data
    X_dense_scaled_noise = scaler_noise.transform(X_dense)
    Y_pred_noise = mlp_noise.predict(X_dense_scaled_noise)
    Y_true_clean = Sin_Square_1D(X_dense)
    
    mse = mean_squared_error(Y_true_clean, Y_pred_noise)
    mse_scores.append(mse)
    
    # Plot results
    plt.subplot(2, 3, i+1)
    plt.plot(X_dense, Y_true_clean, label="True Function", color='black', linewidth=2)
    plt.plot(X_dense, Y_pred_noise, label="MLP Prediction", linestyle='--', alpha=0.8)
    plt.scatter(Xtrain_1D, Ytrain_SS1D_noisy, color='red', alpha=0.6, label="Noisy Training")
    plt.title(f'Noise Level: {noise_level:.2f}\nMSE: {mse:.4f}')
    plt.grid(True, alpha=0.3)
    if i == 0:
        plt.legend()

# Plot MSE vs Noise Level
plt.subplot(2, 3, 6)
plt.plot(noise_levels, mse_scores, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Noise Level')
plt.ylabel('MSE on Clean Data')
plt.title('Model Robustness to Training Noise')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Noise Robustness Results:")
for noise, mse in zip(noise_levels, mse_scores):
    print(f"Noise Level {noise:.2f}: MSE = {mse:.4f}")

## 🎯 Gaussian Process Regression

Let's implement Gaussian Process Regression for function learning and compare it with our neural network approach.

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel

# Define different kernels to test
kernels = {
    'RBF': C(1.0) * RBF(1.0),
    'RBF + Noise': C(1.0) * RBF(1.0) + WhiteKernel(1e-2),
    'RBF (optimized)': C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))
}

plt.figure(figsize=(15, 5))

for i, (kernel_name, kernel) in enumerate(kernels.items()):
    # Create and fit Gaussian Process
    gp = GaussianProcessRegressor(
        kernel=kernel,
        n_restarts_optimizer=10,
        random_state=42
    )
    
    gp.fit(Xtrain_1D, Ytrain_SS1D)
    
    # Make predictions with uncertainty
    Y_pred_gp, Y_std_gp = gp.predict(X_dense, return_std=True)
    
    # Calculate metrics
    Y_true_dense = Sin_Square_1D(X_dense)
    mse_gp = mean_squared_error(Y_true_dense, Y_pred_gp)
    
    # Plot results
    plt.subplot(1, 3, i+1)
    plt.plot(X_dense, Y_true_dense, 'k-', label='True Function', linewidth=2)
    plt.plot(X_dense, Y_pred_gp, 'b-', label='GP Prediction', alpha=0.8)
    plt.fill_between(X_dense.ravel(), 
                     Y_pred_gp - 1.96 * Y_std_gp,
                     Y_pred_gp + 1.96 * Y_std_gp,
                     alpha=0.3, color='blue', label='95% Confidence')
    plt.scatter(Xtrain_1D, Ytrain_SS1D, c='red', s=50, alpha=0.8, label='Training Data')
    plt.title(f'{kernel_name}\nMSE: {mse_gp:.4f}')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend()
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Compare GP with MLP
print("\\nModel Comparison:")
print(f"MLP MSE: {mean_squared_error(Y_true, Y_pred):.4f}")
print(f"GP (RBF + Noise) MSE: {mse_gp:.4f}")
print(f"GP provides uncertainty estimates: ±{Y_std_gp.mean():.3f} (average std)")

## 📊 Model Performance Summary

Let's create a comprehensive comparison of all models and their performance metrics.

In [ ]:
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error

# Create performance comparison
models_performance = []

# Re-calculate metrics for fair comparison
Y_true_comparison = Sin_Square_1D(X_dense)

# MLP Performance
mlp_pred = mlp.predict(scaler.transform(X_dense))
models_performance.append({
    'Model': 'MLP Regressor',
    'MSE': mean_squared_error(Y_true_comparison, mlp_pred),
    'MAE': mean_absolute_error(Y_true_comparison, mlp_pred),
    'R²': r2_score(Y_true_comparison, mlp_pred),
    'Provides Uncertainty': 'No',
    'Training Time': 'Fast'
})

# GP Performance (using last GP model)
gp_pred, gp_std = gp.predict(X_dense, return_std=True)
models_performance.append({
    'Model': 'Gaussian Process',
    'MSE': mean_squared_error(Y_true_comparison, gp_pred),
    'MAE': mean_absolute_error(Y_true_comparison, gp_pred),
    'R²': r2_score(Y_true_comparison, gp_pred),
    'Provides Uncertainty': 'Yes',
    'Training Time': 'Moderate'
})

# Time Series MLP Performance
if 'mlp_ts' in globals():
    models_performance.append({
        'Model': 'Time Series MLP',
        'MSE': test_mse,
        'MAE': mean_absolute_error(y_prey_test, y_pred_test),
        'R²': mlp_ts.score(X_prey_test_scaled, y_prey_test),
        'Provides Uncertainty': 'No',
        'Training Time': 'Fast'
    })

# Create performance DataFrame
df_performance = pd.DataFrame(models_performance)
print("🏆 Model Performance Comparison")
print("=" * 60)
print(df_performance.round(4).to_string(index=False))

# Visualization of model comparison
plt.figure(figsize=(12, 8))

# Plot 1: MSE Comparison
plt.subplot(2, 2, 1)
models = df_performance['Model']
mse_values = df_performance['MSE']
bars = plt.bar(models, mse_values, color=['skyblue', 'lightcoral', 'lightgreen'][:len(models)])
plt.title('Mean Squared Error Comparison')
plt.ylabel('MSE')
plt.xticks(rotation=45)
for bar, value in zip(bars, mse_values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
             f'{value:.4f}', ha='center', va='bottom')

# Plot 2: R² Comparison
plt.subplot(2, 2, 2)
r2_values = df_performance['R²']
bars = plt.bar(models, r2_values, color=['skyblue', 'lightcoral', 'lightgreen'][:len(models)])
plt.title('R² Score Comparison')
plt.ylabel('R²')
plt.xticks(rotation=45)
for bar, value in zip(bars, r2_values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{value:.4f}', ha='center', va='bottom')

# Plot 3: Model Predictions Comparison
plt.subplot(2, 1, 2)
plt.plot(X_dense, Y_true_comparison, 'k-', label='True Function', linewidth=3)
plt.plot(X_dense, mlp_pred, '--', label='MLP Prediction', alpha=0.8, linewidth=2)
plt.plot(X_dense, gp_pred, ':', label='GP Prediction', alpha=0.8, linewidth=2)
if len(gp_std) > 0:
    plt.fill_between(X_dense.ravel(), 
                     gp_pred - 1.96 * gp_std,
                     gp_pred + 1.96 * gp_std,
                     alpha=0.2, color='red', label='GP 95% Confidence')
plt.scatter(Xtrain_1D, Ytrain_SS1D, c='red', s=50, alpha=0.8, label='Training Data')
plt.title('Model Predictions Comparison')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()